Code to collect NED predictions, (previously) fetched by remote thinclient, plus code to run actual 7 days ahead predictions for energy productions.

In [1]:
import pandas as pd

# Read the CSV files
df_NED_preds_CSV_2 = pd.read_csv('../data/ned-api/appended_NED_preds-1May2025.csv')
df_NED_preds_CSV_1 = pd.read_csv('../data/ned-api/appended_NED_preds_2-10-April.csv')


# Convert current_datetime to datetime type if it's not already
df_NED_preds_CSV_2['current_datetime'] = pd.to_datetime(df_NED_preds_CSV_2['current_datetime'])
df_NED_preds_CSV_1['current_datetime'] = pd.to_datetime(df_NED_preds_CSV_1['current_datetime'])



In [2]:
# Analyze date ranges for each DataFrame


print("\nDate ranges in df_NED_preds_CSV_1:")
print(f"Start: {df_NED_preds_CSV_1['current_datetime'].min()}")
print(f"End: {df_NED_preds_CSV_1['current_datetime'].max()}")

print("\nDate ranges in df_NED_preds_CSV_2:")
print(f"Start: {df_NED_preds_CSV_2['current_datetime'].min()}")
print(f"End: {df_NED_preds_CSV_2['current_datetime'].max()}")


Date ranges in df_NED_preds_CSV_1:
Start: 2025-04-02 08:17:40
End: 2025-04-10 12:00:05

Date ranges in df_NED_preds_CSV_2:
Start: 2025-04-10 14:07:18
End: 2025-05-01 00:00:05


In [3]:

# Concatenate both dataframes
df_NED_preds_CSV = pd.concat([df_NED_preds_CSV_1, df_NED_preds_CSV_2], ignore_index=True)

# Drop any potential duplicates based on current_datetime
df_NED_preds_CSV = df_NED_preds_CSV.drop_duplicates(subset=['current_datetime'], keep='first')

# Sort by current_datetime
df_NED_preds_CSV = df_NED_preds_CSV.sort_values('current_datetime')

# Display the date range
print("Date range in the dataset:")
print(f"Start date: {df_NED_preds_CSV['current_datetime'].min()}")
print(f"End date: {df_NED_preds_CSV['current_datetime'].max()}")
print(f"\nTotal number of records: {len(df_NED_preds_CSV)}")

Date range in the dataset:
Start date: 2025-04-02 08:17:40
End date: 2025-05-01 00:00:05

Total number of records: 265


In [4]:
import sqlite3

# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Write the DataFrame to the database table 'raw_NED_preds'
# If table exists, replace it. If not, create new table
df_NED_preds_CSV.to_sql('raw_NED_preds', conn, if_exists='replace', index=False)


record_count = df_NED_preds_CSV.shape[0]
print(f"{record_count} records successfully written to database table 'raw_NED_preds'")
# Close the connection
conn.close()

265 records successfully written to database table 'raw_NED_preds'


In [5]:
# connecting to the SQLite database and checking the most recent date in the raw_NED_preds table

import sqlite3
import pandas as pd


# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)


# collect the date column from the raw_NED_preds table
hist_weather_dates = pd.read_sql_query("SELECT current_datetime FROM raw_NED_preds", conn)
conn.close()

# Convert to datetime and find max date
hist_weather_dates['current_datetime'] = pd.to_datetime(hist_weather_dates['current_datetime'])
most_recent_NED_pred_date = hist_weather_dates['current_datetime'].max().strftime('%Y-%m-%d')
print(f"The most recent date in raw_NED_preds is: {most_recent_NED_pred_date}")



The most recent date in raw_NED_preds is: 2025-05-01
